In [178]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


In [179]:
df = pd.read_csv("matches.csv")

df.head()

/var/folders/x9/fxnv8bb96j99gbr28jn7scdc0000gn/T/ipykernel_10441/3816335167.py:1: DtypeWarning: Columns (136,141,143,145,146,148,149,151,152,154) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("matches.csv")


,id,home,away,date,year,time (utc),attendance,venue,league,part_of_competition,...,home_bench_10,home_bench_10_minute,home_bench_11_num,home_bench_11,home_bench_11_minute,away_bench_10_num,away_bench_10,away_bench_10_minute,away_starting_14_num,away_starting_14
0,18123,Charlton Athletic,Everton,"Saturday, August 18",2001,14:00,"20,451","The Valley, London, England",2001-2002 Barclays Premier League,na,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,18124,Derby County,Blackburn Rovers,"Saturday, August 18",2001,14:00,"28,236","Pride Park Stadium, England",2001-2002 Barclays Premier League,na,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,18125,Leeds United,Southampton,"Saturday, August 18",2001,14:00,"39,715","Elland Road, Leeds, England",2001-2002 Barclays Premier League,na,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,18126,Leicester City,Bolton Wanderers,"Saturday, August 18",2001,14:00,"19,987","Filbert Street, England",2001-2002 Barclays Premier League,na,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,18127,Liverpool,West Ham United,"Saturday, August 18",2001,14:00,"43,935","Anfield, Liverpool, England",2001-2002 Barclays Premier League,na,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [181]:
df['result'] = np.where(df['home_score'] > df['away_score'], 1, np.where(df['home_score'] < df['away_score'], 2, 0))

columnas_deseadas = ['home','away','home_score', 'away_score', 'result']
df = df.filter(items=columnas_deseadas)

filtered_df = df[((df['home'] == 'Liverpool') & (df['away'] == 'Manchester United')) | ((df['home'] == 'Manchester United') & (df['away'] == 'Liverpool'))]

df_encoded = pd.get_dummies(filtered_df, columns=['home', 'away'])

print(df_encoded.head(10))




      home_score  away_score  result  home_Liverpool  home_Manchester United  \
109            3           1       1            True                   False   
228            0           1       2           False                    True   
535            1           2       2            True                   False   
691            4           0       1           False                    True   
876            1           2       2            True                   False   
1101           0           1       2           False                    True   
1198           2           1       1           False                    True   
1360           0           1       2            True                   False   
1571           0           0       0            True                   False   
1742           1           0       1           False                    True   

      away_Liverpool  away_Manchester United  
109            False                    True  
228             True     

In [176]:
X = df_encoded.drop('result', axis=1)
y = df_encoded['result']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [5, 10, 15],
    'criterion': ['gini', 'entropy']
}

rf_classifier = RandomForestClassifier(random_state=42)

grid_search = GridSearchCV(estimator=rf_classifier, param_grid=param_grid, cv=5)
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
best_score = grid_search.best_score_

rf_classifier_best = RandomForestClassifier(random_state=42, **best_params)
rf_classifier_best.fit(X_train, y_train)

y_pred = rf_classifier_best.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

Accuracy: 0.8888888888888888
Precision: 0.9166666666666666
Recall: 0.8888888888888888
F1 Score: 0.8888888888888888
